# Prompt Templates and Variables Tutorial (Using Jinja2)

## Overview

This tutorial provides a comprehensive introduction to creating and using prompt templates with variables in the context of AI language models. It focuses on leveraging Python and the Jinja2 templating engine to create flexible, reusable prompt structures that can incorporate dynamic content. The tutorial demonstrates how to interact with OpenAI's GPT models using these advanced prompting techniques.

## Motivation

As AI language models become increasingly sophisticated, the ability to craft effective prompts becomes crucial for obtaining desired outputs. Prompt templates and variables offer several advantages:

1. **Reusability**: Templates can be reused across different contexts, saving time and ensuring consistency.
2. **Flexibility**: Variables allow for dynamic content insertion, making prompts adaptable to various scenarios.
3. **Complexity Management**: Templates can handle complex structures, including conditional logic and loops, enabling more sophisticated interactions with AI models.
4. **Scalability**: As applications grow, well-structured templates make it easier to manage and maintain large numbers of prompts.

This tutorial aims to equip learners with the knowledge and skills to create powerful, flexible prompt templates, enhancing their ability to work effectively with AI language models.

## Key Components

The tutorial covers several key components:

1. **PromptTemplate Class**: A custom class that wraps Jinja2's Template class, providing a simple interface for creating and using templates.
2. **Jinja2 Templating**: Utilization of Jinja2 for advanced templating features, including variable insertion, conditional statements, and loops.
3. **OpenAI API Integration**: Direct use of the OpenAI API for sending prompts and receiving responses from GPT models.
4. **Variable Handling**: Techniques for incorporating variables into templates and managing dynamic content.
5. **Conditional Logic**: Implementation of if-else statements within templates to create context-aware prompts.
6. **Advanced Formatting**: Methods for structuring complex prompts, including list formatting and multi-part instructions.

## Method Details

The tutorial employs a step-by-step approach to introduce and demonstrate prompt templating concepts:

1. **Setup and Environment**: The lesson begins by setting up the necessary libraries, including Jinja2 and the OpenAI API client.

2. **Basic Template Creation**: Introduction to creating simple templates with single and multiple variables using the custom PromptTemplate class.

3. **Variable Insertion**: Demonstration of how to insert variables into templates using Jinja2's `{{ variable }}` syntax.

4. **Conditional Content**: Exploration of using if-else statements in templates to create prompts that adapt based on provided variables.

5. **List Processing**: Techniques for handling lists of items within templates, including iteration and formatting.

6. **Advanced Templating**: Demonstration of more complex template structures, including nested conditions, loops, and multi-part prompts.

7. **Dynamic Instruction Generation**: Creation of templates that can generate structured instructions based on multiple input variables.

8. **API Integration**: Throughout the tutorial, examples show how to use the templates with the OpenAI API to generate responses from GPT models.

The methods are presented with practical examples, progressing from simple to more complex use cases. Each concept is explained theoretically and then demonstrated with a practical application.

## Conclusion

This tutorial provides a solid foundation in creating and using prompt templates with variables, leveraging the power of Jinja2 for advanced templating features. By the end of the lesson, learners will have gained:

1. Understanding of the importance and applications of prompt templates in AI interactions.
2. Practical skills in creating reusable, flexible prompt templates.
3. Knowledge of how to incorporate variables and conditional logic into prompts.
4. Experience in structuring complex prompts for various use cases.
5. Insight into integrating templated prompts with the OpenAI API.

These skills enable more sophisticated and efficient interactions with AI language models, opening up possibilities for creating more advanced, context-aware AI applications. The techniques learned can be applied to a wide range of scenarios, from simple query systems to complex, multi-turn conversational agents.

## Setup

In [ ]:
import os
from jinja2 import Template
from dotenv import load_dotenv

load_dotenv()

#model="gpt-4o-mini"
#model="tinyllama:latest"
model="llama3.2:1b"
ollama_port=12434
ollama_server=f"http://host.docker.internal:{ollama_port}"

if model.startswith("gpt"):
    import openai
    openai.api_key = os.getenv('OPENAI_API_KEY')
else:
    from ollama import chat
    from ollama import Client
    ollama_client = Client(host=ollama_server)
    # Verify server is reachable and model is available
    try:
        available = ollama_client.list()
        model_names = [m.model for m in (available.models or []) if m.model]
        if model not in model_names:
            raise ValueError(
                f"Model '{model}' not found at {ollama_server}. "
                f"Available: {', '.join(model_names) or 'none'}. "
                f"Pull with: ollama pull {model}"
            )
        print(f"Using Ollama at {ollama_server} with model '{model}'.")
    except Exception as e:
        if "Connection" in type(e).__name__ or "connect" in str(e).lower():
            raise ConnectionError(
                f"Cannot reach Ollama at {ollama_server}. "
                f"Is Ollama running on port {ollama_port}?"
            ) from e
        raise

def get_completion(prompt, model=model):
    ''' Get a completion from the OpenAI API 
    Args:
        prompt (str): The prompt to send to the API
        model (str): The model to use for the completion
    Returns:
        str: The completion text
    '''
    messages = [{"role": "user", "content": prompt}]
    if model.startswith("gpt"):
        response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0,
        )
        return response.choices[0].message["content"]
    else:
        response = ollama_client.chat(
            model=model,
            messages=messages,
            options={"temperature": 0},
        )
        return response.message.content


## 1. Creating Reusable Prompt Templates

We'll create a PromptTemplate class that uses Jinja2 for templating:

In [ ]:
class PromptTemplate:
    ''' A class to represent a template for generating prompts with variables
    Attributes:
        template (str): The template string with variables
        input_variables (list): A list of the variable names in the template
    '''
    def __init__(self, template, input_variables):
        self.template = Template(template)
        self.input_variables = input_variables
    
    def format(self, **kwargs):
        return self.template.render(**kwargs)

# Simple template with one variable
simple_template = PromptTemplate(
    template="Provide a brief explanation of {{ topic }}.",
    input_variables=["topic"]
)

# More complex template with multiple variables
complex_template = PromptTemplate(
    template="Explain the concept of {{ concept }} in the field of {{ field }} to a {{ audience }} audience, concisely.",
    input_variables=["concept", "field", "audience"]
)


In [ ]:

# Using the simple template
print("Simple Template Result:")
prompt = simple_template.format(topic="photosynthesis")
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

# Using the complex template
print("Complex Template Result (beginner):")
prompt = complex_template.format(
    concept="neural networks",
    field="artificial intelligence",
    audience="beginner"
)
print(get_completion(prompt))

# Using the complex template
print("Complex Template Result (baby):")
prompt = complex_template.format(
    concept="neural networks",
    field="artificial intelligence",
    audience="baby"
)
print(get_completion(prompt))

## 2. Using Variables for Dynamic Content

Now let's explore more advanced uses of variables, including conditional content:

In [ ]:
# Template with conditional content
conditional_template = PromptTemplate(
    template="My name is {{ name }} and I am {{ age }} years old. "
              "{% if profession %}I work as a {{ profession }}.{% else %}I am currently not employed.{% endif %} "
              "Can you give me career advice based on this information? answer concisely.",
    input_variables=["name", "age", "profession"]
)

# Using the conditional template
print("Conditional Template Result (with profession):")
prompt = conditional_template.format(
    name="Alex",
    age="28",
    profession="software developer"
)
print(get_completion(prompt))

print("\nConditional Template Result (without profession):")
prompt = conditional_template.format(
    name="Sam",
    age="22",
    profession=""
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")


In [ ]:
# Template for list processing
list_template = PromptTemplate(
    template="Categorize these items into groups: {{ items }}. Provide the categories and the items in each category.",
    input_variables=["items"]
)

# Using the list template
print("List Template Result:")
prompt = list_template.format(
    items="apple, banana, carrot, hammer, screwdriver, pliers, novel, textbook, magazine"
)
print(get_completion(prompt))

## Advanced Template Techniques

Let's explore some more advanced techniques for working with prompt templates and variables:

In [5]:
# Template with formatted list
list_format_template = PromptTemplate(
    template="Analyze the following list of items:\n"
              "{% for item in items.split(',') %}"
              "- {{ item.strip() }}\n"
              "{% endfor %}"
              "\nProvide a summary of the list and suggest any patterns or groupings.",
    input_variables=["items"]
)


# Using the formatted list template
print("Formatted List Template Result:")
prompt = list_format_template.format(
    items="Python, JavaScript, HTML, CSS, React, Django, Flask, Node.js"
)
print(get_completion(prompt))

print("\n" + "-"*50 + "\n")

Formatted List Template Result:
After analyzing the list, I've identified some patterns and groupings that can be useful for understanding the relationships between these programming languages.

**Grouping by Category**

1. **Web Development Frameworks**
	* Python (Django, Flask)
	* JavaScript (React)
2. **Front-end Frameworks**
	* HTML
	* CSS
3. **Back-end Frameworks**
	* Node.js

**Patterns and Observations**

1. **JavaScript is a popular choice for front-end development**: React is a well-known library for building user interfaces, while Node.js is often used for server-side development.
2. **Python and Django are commonly used for back-end development**: Both frameworks are popular choices for building web applications with complex backend logic.
3. **HTML and CSS are fundamental to web development**: These two technologies are essential for building the structure and layout of a website or application.
4. **Node.js is often used for real-time applications**: Its event-driven, non-

In [6]:
# Template with dynamic instructions
dynamic_instruction_template = PromptTemplate(
    template="Task: {{ task }}\n"
              "Context: {{ context }}\n"
              "Constraints: {{ constraints }}\n\n"
              "Please provide a solution that addresses the task, considers the context, and adheres to the constraints.",
    input_variables=["task", "context", "constraints"]
)

# Using the dynamic instruction template
print("Dynamic Instruction Template Result:")
prompt = dynamic_instruction_template.format(
    task="Design a logo for a tech startup. If you are not able to design a logo, just describe what you would do.",
    context="The startup focuses on AI-driven healthcare solutions",
    constraints="Must use blue and green colors, and should be simple enough to be recognizable when small"
)
print(get_completion(prompt))

Dynamic Instruction Template Result:
Based on the context and constraints, I would design a logo for the tech startup focused on AI-driven healthcare solutions using blue and green colors. Here's a potential solution:

**Logo Concept:**

The logo features a stylized, interconnected "A" made up of two curved lines that resemble a wave or a waveform. The lines are designed to evoke the idea of data flowing through the system, while also being simple and recognizable.

**Color Scheme:**

* Primary color: A deep blue (#212121) used for the background and accents.
* Secondary color: A bright green (#34C759) used for highlights and text.

**Logo Variations:**

To ensure versatility, I would create a few logo variations:

1. **Main Logo:** The primary logo featuring the stylized "A" with blue and green colors.
2. **Icon-only Logo:** A simplified version of the "A" without the background or waveform, used for social media profiles, app icons, or other small-scale applications.
3. **Text-only L